In [17]:
from orderbook_copy import OrderBook
import pandas as pd
book = OrderBook()

In [18]:
def dispatch(action, oid, side, price, size):
    a = (action or "").strip().lower()
    if a in ("a", "add"):       book.add_order(oid, side, float(price), int(size))
    elif a in ("c", "cancel"):  book.cancel_order(oid)
    elif a in ("m", "modify"):  book.modify_order(oid, int(size))
    elif a in ("f", "fill"):    book.fill_order(oid, int(size))

In [19]:
df = pd.read_parquet("MBO.parquet")
df.head()

,ts_event,rtype,publisher_id,instrument_id,action,side,price,size,channel_id,order_id,flags,ts_in_delta,sequence,symbol
ts_recv,,,,,,,,,,,,,,
2024-01-01 00:00:00+00:00,2023-12-31 13:00:04.466568089+00:00,160,1,38601,R,N,NaN,0,26,0,40,0,0,CLF5
2024-01-01 00:00:00+00:00,2023-12-31 13:00:04.466568089+00:00,160,1,38601,A,B,68.0,1,26,8048898538378,40,0,50447,CLF5
2024-01-01 00:00:00+00:00,2023-12-31 13:00:04.466568089+00:00,160,1,38601,A,B,67.0,1,26,8048898527545,40,0,50447,CLF5
2024-01-01 00:00:00+00:00,2023-12-31 13:00:04.466568089+00:00,160,1,38601,A,B,66.5,1,26,8048898560786,40,0,50447,CLF5
2024-01-01 00:00:00+00:00,2023-12-31 13:00:04.466568089+00:00,160,1,38601,A,B,66.0,1,26,8048898569034,40,0,50447,CLF5


In [20]:
required = {"action", "order_id", "side", "price", "size"}
missing = required.difference(df.columns)
if missing:
    raise ValueError(f"Missing required columns: {sorted(missing)}")

for row in df.itertuples(index=False):
    act = row.action
    # skip refresh or unknown
    if act not in ("A","C","M","F","Add","Cancel","Modify","Fill"):
        continue
    # guard NaNs (common in refresh rows)
    if pd.isna(row.price) or pd.isna(row.size):
        continue

    dispatch(act, str(row.order_id), row.side, row.price, row.size)

In [21]:
def _peek_best(sd, want_max=False):
    if not sd: return None
    return sd.peekitem(-1 if want_max else 0)

best_bid = _peek_best(book.bids, want_max=True)
best_ask = _peek_best(book.asks, want_max=False)

print("\n=== TOP OF BOOK ===")
if best_bid:
    px, lvl = best_bid
    print(f"Best Bid: {px:.4f} x {lvl.total_size} (orders={lvl.order_count})")
else:
    print("Best Bid: (empty)")

if best_ask:
    px, lvl = best_ask
    print(f"Best Ask: {px:.4f} x {lvl.total_size} (orders={lvl.order_count})")
else:
    print("Best Ask: (empty)")

print("\n=== GLOBAL STATS ===")
print(f"Bids: total_size={book.bid_stats.total_size}, total_orders={book.bid_stats.total_count}")
print(f"Asks: total_size={book.ask_stats.total_size}, total_orders={book.ask_stats.total_count}")


=== TOP OF BOOK ===
Best Bid: 69.5600 x 1 (orders=1)
Best Ask: 70.6400 x 1 (orders=1)

=== GLOBAL STATS ===
Bids: total_size=10, total_orders=10
Asks: total_size=6, total_orders=6
